In [7]:
from dotenv import load_dotenv
from pathlib import Path

load_dotenv(override=True)
base_dir = Path().resolve().parent

In [8]:
from langchain_openai import ChatOpenAI

llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0, model=llm_model)

In [9]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)

chunk_size = 1000
chunk_overlap = 200

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", " ", ""],
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator="\n"
)

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(f"{base_dir}/data/CourseOutline.pdf")
pages = loader.load()
len(pages)

4

In [11]:
spitted_docs = r_splitter.split_documents(pages)
len(spitted_docs)

9

In [12]:
persist_directory_name = "persist_vectorstore"
persist_directory = Path.joinpath(base_dir, persist_directory_name)
persist_directory_str = persist_directory.as_posix()
persist_directory_str

'/home/zhbdripon/Documents/document-chat/persist_vectorstore'

In [ ]:
!rm -rf ../peripersist_vectorstore

In [14]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb = Chroma.from_documents(
    documents=spitted_docs, embedding=embedding, persist_directory=persist_directory_str
)

In [ ]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# Reusing the persistant
vectordb = Chroma(embedding_function=embedding, persist_directory=persist_directory_str)

In [ ]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [15]:
retriever = vectordb.as_retriever(
    search_type="mmr", search_kwargs={"k": 4, "fetch_k": 20, "lambda_mult": 0.5}
)

In [27]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage
from pydantic import BaseModel, Field


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: list[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


history = get_by_session_id("1")
history.add_message(AIMessage(content="hello"))
print(store)  # noqa: T201

{'1': InMemoryHistory(messages=[AIMessage(content='hello', additional_kwargs={}, response_metadata={})])}


In [17]:
from langchain_core.prompts import PromptTemplate

rag_prompt = PromptTemplate.from_template(
    """You are a helpful assistant answering questions about the following context.
If the question is a follow-up, use the chat history to interpret it.

Chat History:
{history}

Context:
{context}

Question: {question}
Answer:"""
)

In [18]:
refine_prompt = PromptTemplate.from_template(
    """You are a helpful assistant that rephrases follow-up questions into standalone questions.

Chat History:
{history}

Follow-up Question:
{question}

Rewritten standalone question:"""
)

In [19]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser


rewrite_question = refine_prompt | llm | StrOutputParser()

question_refinement_chain = RunnableLambda(
    lambda x: {
        "rewritten": rewrite_question.invoke(
            {"question": x["question"], "history": x["history"]}
        ),
        "history": x["history"],
    }
) | RunnableLambda(
    lambda x: {"question": x["rewritten"].strip(), "history": x["history"]}
)

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | retriever | format_docs,
        "question": RunnableLambda(lambda x: x["question"]),
        "history": RunnableLambda(lambda x: x["history"]),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [21]:
full_chain = question_refinement_chain | rag_chain

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    full_chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)